In [1]:
import numpy as np
import pandas as pd
import re

Сколько страниц из русской Википедии в датасете? - **102**

Какая самая популярная страница русской Википедии (в среднем)? - **Facebook**

Значение SMAPE для предсказаний на основе последнего дня: **54.16127748085961**

Значение SMAPE для предсказаний на основе последних 30 дней: **52.465883293370794**

Минимальное значение SMAPE для предсказаний на основе медианы за последние **27** дней: **49.83547039727196**

In [2]:
train = pd.read_csv('wikipedia_train.csv')
test = pd.read_csv('wikipedia_test.csv')

In [3]:
def pandas_smape(df):
    df.fillna(0, inplace=True)
    df["SMAPE"] = 200 * np.abs(df["Visits"] - df["pred_Visits"]) / (df["Visits"] + df["pred_Visits"])
    df["SMAPE"].fillna(0, inplace=True)
    return np.mean(df["SMAPE"])

In [4]:
train_melted = pd.melt(train, id_vars=['Page'], value_vars=train.columns[1:], var_name='date', value_name='Visits')
test_melted = pd.melt(test, id_vars=['Page'], value_vars=test.columns[1:], var_name='date', value_name='Visits')
print('shape of train_melted: {}'.format(train_melted.shape))
print('shape of test_melted: {}'.format(test_melted.shape))

shape of train_melted: (401892, 3)
shape of test_melted: (58218, 3)


Last day baseline
(Нужно сделать прогноз на основе посещений в последний известный нам день из train (продублировать значение для каждого дня в test))

In [5]:
date = train.columns[-1]
visits = train[date]
visits = pd.concat([visits] * 62, ignore_index=True)

test_melted['pred_Visits'] = visits

print('SMAPE base on last day: {}'.format(pandas_smape(test_melted)))

SMAPE base on last day: 54.16127748085961


Нужно сделать прогноз на основе медианы за последние 30 дней из train.

In [6]:
visits_30 = train[train.columns[-30:]].median(axis=1)
visits_30 = pd.concat([visits_30] * 62, ignore_index=True)

test_melted['pred_Visits'] = visits_30

print('SMAPE base on 30 last days: {}'.format(pandas_smape(test_melted)))

SMAPE base on 30 last days: 52.465883293370794


In [71]:
days = ['{:02d}'.format(i) for i in range(1, 32)]


for day in days:
    
    cols = [col for col in train.columns if col[-2:] == day]

    visit_day_mean = train.loc[:, cols].mean(axis=1)
    
    n = int(len(test_melted[test_melted.date.str[-2:] == day]) / 939)
    
    result = np.array(pd.concat([visit_day_mean] * n, ignore_index=True))
    
    test_melted.loc[test_melted.date.str[-2:] == day, 'pred_Visits'] = result
    

    
print('SMAPE : {}'.format(pandas_smape(test_melted)))

SMAPE : 61.99148730061877


In [162]:
test_melted.date = pd.to_datetime(test_melted.date)

test_melted['day_of_week'] = test_melted.date.dt.weekday_name

days = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']

# result_dict = dict()

# for size in range(62, 0, -1):

#     for day in days:

#         cols = [item for item in train.columns[1:] if pd.to_datetime(item).weekday_name == day]

#         cols = cols[-size: ]

#         visit_day_mean = train.loc[:, cols].median(axis=1)

#         n = int(len(test_melted[test_melted.date.dt.weekday_name == day]) / 939)

#         result = np.array(pd.concat([visit_day_mean] * n, ignore_index=True))

#         test_melted.loc[test_melted.date.dt.weekday_name == day, 'pred_Visits'] = result
    
#     smape = pandas_smape(test_melted)
    
#     result_dict[size] = smape
          
#     print('size = {} last days; SMAPE : {}'.format(size, smape))

min_value = min(list(result_dict.values()))

for key, value in result_dict.items():
    if value == min_value:
        day = key
        
print('Min SMAPE = {}\nfor window in {} last days'.format(min_value, day))


Min SMAPE = 49.83547039727196
for window in 27 last days
